In [4]:
# google 'new ipynb kernal for vscode venv'


# get dataframes of flood levels
from helper_methods import *
xml_root = get_xml_root('https://water.weather.gov/ahps2/hydrograph_to_xml.php?gage=rmdv2&output=xml')
forecast_data = get_flood_data(xml_root, 'forecast')
observed_data = get_flood_data(xml_root, 'observed')

